In [33]:
import re
'''import bs4 as bs  
import urllib.request  

scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')  
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:  
    article_text += p.text '''

f = open('1.txt','r')
article_text =f.read()



In [34]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text) 
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

In [35]:
article_text

"Pressured by senior management and boards of directors for more frequent and detailed internal audit and risk assessment information, many audit and risk executives are evaluating their reliance on office productivity tools such as word processing programs and spreadsheets as their primary solution for managing internal audit processes. The decision by internal auditors to use spreadsheets is not surprising, given the pervasiveness, ease of use, low cost, and a user’s general familiarity with the application. While it is easy to create some light-weight solutions using personal productivity tools, many leading audit organizations have found that in the long run, spreadsheet-based solutions become part of the problem rather than part of the solution. Using spreadsheets to manage the internal audit functions presents two considerable problems. The first challenge stems from the inherently risky nature of spreadsheets; their usage poses a significant vulnerability from unauthorized acces

In [36]:
import nltk
sentence_list = nltk.sent_tokenize(article_text)  

In [37]:
import pandas as pd
clean_sentences = pd.Series(sentence_list).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]


In [38]:
word_embeddings = {}
f = open('C:/users/Videet/1/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [39]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [40]:
sim_mat = np.zeros([len(sentence_list), len(sentence_list)])


In [41]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentence_list)):
    for j in range(len(sentence_list)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [42]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_list)), reverse=True)
for i in range(7):
    print(ranked_sentences[i][1])

Just as financial accounting software must have fields and forms to capture financial information, software for capturing information about risks and controls must have a basic structure.
Spreadsheets kill collaboration, work flow and accountability: A central requirement of enterprise audit software is the ability to assign owners to processes, risks, controls, and compliance policies.
The real role of internal audit is to drive up the quantity and quality of reliable information on the status of risks and controls and to make the data available continuously to all key stakeholders.
A well integrated solution provides a common set of functionality for each internal audit process owner with shared functionality for common activities such as risk assessment, process documentation and issue tracking.
For example, financial management requires clear, consistent accounting policies to determine what gets in the books, as well as sophisticated financial systems to capture, manage, analyze a